In [69]:
from typing import Dict, List, Optional, Union
import logging 

import inspect 
from collections import ChainMap
from pprint import pprint

from marshmallow import fields, Schema as MarshMallowSchema

import datetime as dt
import uuid 
import decimal 


logger = logging.getLogger(__name__)


TYPE_MAPPING = {
    str: fields.String,
    float: fields.Float,
    bool: fields.Boolean,
    int: fields.Integer,

    uuid.UUID: fields.UUID,
    decimal.Decimal: fields.Decimal,

    dt.datetime: fields.DateTime,
    dt.time: fields.Time,
    dt.date: fields.Date,
    dt.timedelta: fields.TimeDelta,

    # tuple: fields.Raw,
    # list: fields.List,
    # List: fields.List,
    # set: fields.Raw,
}

def merge_dicts(*dicts):
    merged = dict(dicts[0])
    for d in dicts[1:]: 
        for key, value in d.items():
            merged[key] = value
    return merged


def get_class_fields(cls):
    merged = merge_dicts(
        get_fields_from_annotations(cls),
        get_fields_from_values(cls)
    )
    return merged


def get_fields_from_annotations(cls):
    annotations = dict(inspect.getmembers(cls)).get('__annotations__', {}).items()
    
    return {
        attr_name: TYPE_MAPPING.get(attr_type)()
        for attr_name, attr_type
        in annotations
        if TYPE_MAPPING.get(attr_type) is not None 
    }


def get_fields_from_values(cls):
    return {
        attr_name: attr_type
        for attr_name, attr_type in 
        dict(inspect.getmembers(cls))['__dict__'].items()
        if isinstance(attr_type, fields.Field)
    }


class MetaSchema(type):
    def __new__(cls, clsname, bases, dct):
        newclass = super().__new__(cls, clsname, bases, dct)
        setattr(
            newclass, 
            "schema", 
            type(f'{clsname}Schema', (MarshMallowSchema, ), get_class_fields(newclass))
        )
        return newclass

    
class Schema(metaclass=MetaSchema):
    schema: Optional[MarshMallowSchema] = None

In [22]:
class Response(Schema):
    a: fields.Integer()
    b = None
    c = fields.Integer()

In [66]:
Optional[int].__args__ == (int, type(None))

True

In [67]:
Optional[int] == Optional[Generic]

TypeError: Plain <class 'typing.Generic'> is not valid as type argument

In [75]:
Optional[int]

typing.Union[int, NoneType]

In [76]:
Optional[float]

typing.Union[float, NoneType]

In [77]:
Union[int, type(None)] ==  Union[float, type(None)] 

False

In [79]:
Union[int, type(None)]

typing.Union[int, NoneType]